In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 33.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (4

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizer, AutoModel, AdamW, BertForSequenceClassification

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Satria Data/satria_data_clean.csv")
df.head()

,label,text_clean
0,Sumber Daya Alam,kunjung prabowo resmi serah proyek bantu air b...
1,Politik,anies tepuk tangan riah rektor wajib mata kuli...
2,Demografi,sih dukung 01 goblok dukung 02 ridwan kamil sk...
3,Politik,anies sikap kritis kerja prabowo anggap sopan ...
4,Politik,anies baswedan harap asn tni polri pegang sump...


In [ ]:
df.label.value_counts()

label
Politik                    2969
Sosial Budaya               584
Pertahanan dan Keamanan     400
Ideologi                    400
Ekonomi                     366
Sumber Daya Alam            191
Demografi                    62
Geografi                     19
Name: count, dtype: int64

In [ ]:
data = df[['text_clean', 'label']]
data['encoded_label'] = df['label'].astype('category').cat.codes
data.head()

<ipython-input-6-8c30771db6ee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['encoded_label'] = df['label'].astype('category').cat.codes


,text_clean,label,encoded_label
0,kunjung prabowo resmi serah proyek bantu air b...,Sumber Daya Alam,7
1,anies tepuk tangan riah rektor wajib mata kuli...,Politik,5
2,sih dukung 01 goblok dukung 02 ridwan kamil sk...,Demografi,0
3,anies sikap kritis kerja prabowo anggap sopan ...,Politik,5
4,anies baswedan harap asn tni polri pegang sump...,Politik,5


In [ ]:
category_mapping = df['label'].astype('category').cat.categories

# Print the mapping
print("Category to code mapping:")
for code, category in enumerate(category_mapping):
    print(f"{category}: {code}")

Category to code mapping:
Demografi: 0
Ekonomi: 1
Geografi: 2
Ideologi: 3
Pertahanan dan Keamanan: 4
Politik: 5
Sosial Budaya: 6
Sumber Daya Alam: 7


In [ ]:
data_texts = data['text_clean'].to_list()

data_labels = data['encoded_label'].to_list()

In [ ]:
df_train,df_val = train_test_split(data, test_size = 0.2, random_state = 42 )

df_val,df_test = train_test_split(df_val, test_size = 0.1, random_state = 42 )

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('indobenchmark/indobert-base-p1')

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples['text_clean'], padding='max_length', truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3992 [00:00<?, ? examples/s]

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'encoded_label'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'encoded_label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'encoded_label'])

In [ ]:
train_dataset = train_dataset.rename_column("encoded_label", "labels")
val_dataset = val_dataset.rename_column("encoded_label", "labels")
test_dataset = test_dataset.rename_column("encoded_label", "labels")

In [ ]:
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(data['label'].unique()))
model = BertForSequenceClassification.from_pretrained('indobenchmark/indobert-base-p1', num_labels=len(data['label'].unique()))

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from sklearn.metrics import balanced_accuracy_score, f1_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {
        'balanced_accuracy': balanced_accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='weighted')
    }

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Balanced Accuracy,F1
1,0.675600,0.748727,0.484581,0.761838
2,0.464200,0.705709,0.500869,0.778059


Epoch,Training Loss,Validation Loss,Balanced Accuracy,F1
1,0.675600,0.748727,0.484581,0.761838
2,0.464200,0.705709,0.500869,0.778059
3,0.372000,0.777081,0.515289,0.786039
4,0.255400,0.878860,0.515704,0.780779
5,0.237200,1.041263,0.519724,0.783093
6,0.177700,1.074226,0.512639,0.784790
7,0.225700,1.123540,0.530891,0.794994
8,0.068600,1.182428,0.524375,0.785143
9,0.005300,1.187846,0.526467,0.793187
10,0.089800,1.184471,0.529540,0.796529


TrainOutput(global_step=2500, training_loss=0.2785441485106945, metrics={'train_runtime': 3737.472, 'train_samples_per_second': 10.681, 'train_steps_per_second': 0.669, 'total_flos': 1.050395916435456e+16, 'train_loss': 0.2785441485106945, 'epoch': 10.0})

In [ ]:
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 1.1844708919525146, 'eval_balanced_accuracy': 0.5295398195222165, 'eval_f1': 0.7965285750290702, 'eval_runtime': 25.3929, 'eval_samples_per_second': 35.404, 'eval_steps_per_second': 2.245, 'epoch': 10.0}


In [ ]:
model.save_pretrained('/content/drive/MyDrive/Satria Data/result_stemstop')
tokenizer.save_pretrained('/content/drive/MyDrive/Satria Data/result_stemstop')

('/content/drive/MyDrive/Satria Data/result_stemstop/tokenizer_config.json',
 '/content/drive/MyDrive/Satria Data/result_stemstop/special_tokens_map.json',
 '/content/drive/MyDrive/Satria Data/result_stemstop/vocab.txt',
 '/content/drive/MyDrive/Satria Data/result_stemstop/added_tokens.json')

In [ ]:
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Satria Data/result_stemstop')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Satria Data/result_stemstop')

# Tokenize the test dataset (if not already tokenized)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define a new Trainer for evaluation
test_trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Evaluate the model on the test dataset
test_results = test_trainer.evaluate(test_dataset)
print(f"Test results: {test_results}")

# Making predictions on the test dataset
predictions = test_trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)

# Convert predictions and true labels to a pandas DataFrame for better readability
pred_df = pd.DataFrame({
    'predictions': preds,
    'true_labels': test_dataset['labels']
})

print(pred_df)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Test results: {'eval_loss': 1.1749848127365112, 'eval_balanced_accuracy': 0.7085911784068466, 'eval_f1': 0.813916778305525, 'eval_runtime': 3.2396, 'eval_samples_per_second': 30.868, 'eval_steps_per_second': 2.161}
    predictions  true_labels
0             1            1
1             5            5
2             5            5
3             5            5
4             5            5
..          ...          ...
95            5            5
96            3            3
97            3            3
98            5            5
99            3            3

[100 rows x 2 columns]


In [ ]:
# Calculate the balanced accuracy
true_labels = test_dataset['labels']
balanced_accuracy = balanced_accuracy_score(true_labels, preds)
print(f"Test Balanced Accuracy: {balanced_accuracy}")

Test Balanced Accuracy: 0.7085911784068466


In [ ]:
df_unlabel = pd.read_csv("/content/unlabel_praproses.csv")
df_unlabel.head()

,IDText,Text,clean_data,clean_data2,nostopword_text,stemming_text,stemming_nostopword_text
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...,lu mau org2 prodemokrasi di negara ini bisa pu...,lu mau orang prodemokrasi di negara ini bisa p...,lu orang prodemokrasi negara kesempatan ngelaw...,lu mau orang prodemokrasi di negara ini bisa p...,lu orang prodemokrasi negara sempat ngelawan o...
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...,prabowo ditanya soal hutang luar negeri dia me...,prabowo ditanya soal hutang luar negeri dia me...,prabowo hutang negeri hutang negeri indonesia ...,prabowo tanya soal hutang luar negeri dia jawa...,prabowo hutang negeri hutang negeri indonesia ...
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...,kikidaliyo ganjar pranowo itulah beliau sosok ...,kikidaliyo ganjar pranowo itulah beliau sosok ...,kikidaliyo ganjar pranowo beliau sosok mengagu...,kikidaliyo ganjar pranowo itu beliau sosok yan...,kikidaliyo ganjar pranowo beliau sosok mengagu...
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...,prabowo gibran yang bisa melakukan itu semua d...,prabowo gibran yang bisa melakukan itu semua d...,prabowo gibran kesejahteraan rakyat,prabowo gibran yang bisa laku itu semua demi s...,prabowo gibran sejahtera rakyat
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...,lah justru yg gak nyambung junjungan elu aomkm...,lah justru yang tidak nyambung junjungan elu a...,nyambung junjungan elu aomkmkmkmk ngomongin et...,lah justru yang tidak nyambung junjung elu aom...,nyambung junjung elu aomkmkmkmk ngomongin etik...


In [ ]:
data_texts = df_unlabel['stemming_nostopword_text']

In [ ]:
tokenized_texts = tokenizer(
    data_texts.tolist(),
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='pt'
)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDataset
dataset = TensorDataset(tokenized_texts['input_ids'], tokenized_texts['attention_mask'])

# Create DataLoader
batch_size = 16  # Adjust batch size as needed
dataloader = DataLoader(dataset, batch_size=batch_size)

In [ ]:
# Put model in evaluation mode
model.eval()

# Move model to the appropriate device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

all_predictions = []

with torch.no_grad():
    for batch in dataloader:
        input_ids, attention_masks = [tensor.to(device) for tensor in batch]

        outputs = model(input_ids, attention_mask=attention_masks)
        logits = outputs.logits

        predictions = torch.argmax(logits, dim=-1)
        all_predictions.extend(predictions.cpu().numpy())

# Convert predictions to a pandas DataFrame if needed
predictions_df = pd.DataFrame(all_predictions, columns=['predictions'])

# Optionally, you can merge the predictions with the original dataframe
#result_df = df.copy()
#result_df['predictions'] = predictions_df

In [ ]:
predictions_df.shape

(1000, 1)

In [ ]:
import pandas as pd

# Assuming df is your original dataframe and predictions_df is the dataframe with predictions
# Ensure that predictions_df has the same index as df
predictions_df.index = df_unlabel.index

# Concatenate the original dataframe with the predictions dataframe
result_df = pd.concat([df_unlabel, predictions_df], axis=1)

# Display the result
result_df.head()

,IDText,Text,clean_data,clean_data2,nostopword_text,stemming_text,stemming_nostopword_text,predictions
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...,lu mau org2 prodemokrasi di negara ini bisa pu...,lu mau orang prodemokrasi di negara ini bisa p...,lu orang prodemokrasi negara kesempatan ngelaw...,lu mau orang prodemokrasi di negara ini bisa p...,lu orang prodemokrasi negara sempat ngelawan o...,5
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...,prabowo ditanya soal hutang luar negeri dia me...,prabowo ditanya soal hutang luar negeri dia me...,prabowo hutang negeri hutang negeri indonesia ...,prabowo tanya soal hutang luar negeri dia jawa...,prabowo hutang negeri hutang negeri indonesia ...,5
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...,kikidaliyo ganjar pranowo itulah beliau sosok ...,kikidaliyo ganjar pranowo itulah beliau sosok ...,kikidaliyo ganjar pranowo beliau sosok mengagu...,kikidaliyo ganjar pranowo itu beliau sosok yan...,kikidaliyo ganjar pranowo beliau sosok mengagu...,3
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...,prabowo gibran yang bisa melakukan itu semua d...,prabowo gibran yang bisa melakukan itu semua d...,prabowo gibran kesejahteraan rakyat,prabowo gibran yang bisa laku itu semua demi s...,prabowo gibran sejahtera rakyat,5
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...,lah justru yg gak nyambung junjungan elu aomkm...,lah justru yang tidak nyambung junjungan elu a...,nyambung junjungan elu aomkmkmkmk ngomongin et...,lah justru yang tidak nyambung junjung elu aom...,nyambung junjung elu aomkmkmkmk ngomongin etik...,5


In [ ]:
data_akhir = result_df.drop(columns=['Text', 'clean_data', 'nostopword_text', 'stemming_text', 'stemming_nostopword_text'], axis =1)

In [ ]:
data_akhir = data_akhir.drop(columns=['clean_data2'], axis =1)

data_akhir.head()

,IDText,predictions
0,TXT0001,5
1,TXT0002,5
2,TXT0003,3
3,TXT0004,5
4,TXT0005,5


In [ ]:
data_akhir['predictions'] = data_akhir['predictions'].replace({0: 'Demografi', 1: 'Ekonomi', 2: 'Geografi',
                                           3: 'Ideologi', 4: 'Pertahanan dan Keamanan', 5: 'Politik',
                                           6: 'Sosial Budaya', 7: 'Sumber Daya Alam'})

In [ ]:
data_akhir.head(10)

,IDText,predictions
0,TXT0001,Politik
1,TXT0002,Politik
2,TXT0003,Ideologi
3,TXT0004,Politik
4,TXT0005,Politik
5,TXT0006,Politik
6,TXT0007,Politik
7,TXT0008,Politik
8,TXT0009,Politik
9,TXT0010,Ekonomi


In [ ]:
data_akhir = data_akhir.rename(columns={'predictions' : 'Kelas'})

In [ ]:
data_akhir.head(20)

,IDText,Kelas
0,TXT0001,Politik
1,TXT0002,Politik
2,TXT0003,Ideologi
3,TXT0004,Politik
4,TXT0005,Politik
5,TXT0006,Politik
6,TXT0007,Politik
7,TXT0008,Politik
8,TXT0009,Politik
9,TXT0010,Ekonomi


In [ ]:
data_akhir.Kelas.value_counts()

Kelas
Politik                    766
Ekonomi                    103
Pertahanan dan Keamanan     52
Sosial Budaya               39
Ideologi                    20
Sumber Daya Alam            17
Demografi                    3
Name: count, dtype: int64

In [ ]:
data_akhir.to_csv('SD2024040000390_v3.csv', index=False, sep=',')